In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
labelencoder = LabelEncoder()
data['Gender'] = labelencoder.fit_transform(data['Gender'])


In [5]:
onehotencoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehotencoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehotencoder_geo.get_feature_names_out(['Geography']))

In [6]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [7]:
data = pd.concat([data.drop('Geography', axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
x = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [9]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [10]:
scalar = StandardScaler()
xtrain = scalar.fit_transform(xtrain)
xtest = scalar.transform(xtest) 

In [13]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(labelencoder,file)

with open('onehor_encoded_geo.pkl','wb') as file:
    pickle.dump(onehotencoder_geo, file)

with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)

In [14]:
## train with regression problem stmt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(xtrain.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)  ## default activation fuction is linear activation fn
])

d:\projects\P_1\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "regression/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboardcallback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [22]:
earlystopping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [24]:
history = model.fit(
    xtrain,ytrain,
    validation_data=(xtest,ytest),
    epochs = 100,
    callbacks = [earlystopping, tensorboardcallback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 99866.3828 - mae: 99866.3828 - val_loss: 98581.1016 - val_mae: 98581.1016
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99032.4375 - mae: 99032.4375 - val_loss: 97407.5078 - val_mae: 97407.5078
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 97621.3906 - mae: 97621.3906 - val_loss: 94296.0625 - val_mae: 94296.0625
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 95296.7109 - mae: 95296.7109 - val_loss: 88965.3125 - val_mae: 88965.3125
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 89243.1328 - mae: 89243.1328 - val_loss: 81817.2734 - val_mae: 81817.2734
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 80808.1641 - mae: 80808.1641 - val_loss: 73739.6406 - val_mae: 73739.6406
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 74455.7344 - mae: 74455.7344 - val_loss: 66059.2578 - val_mae: 66059.2578
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - 

In [25]:
## evaluate the model
%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 39060), started 0:00:27 ago. (Use '!kill 39060' to kill it.)

In [28]:
test_loss,test_mae = model.evaluate(xtest,ytest)
print(test_loss,test_mae)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 51161.8008 - mae: 51161.8008
50263.46875 50263.46875


In [29]:
model.save('regression_model.h5')